<a href="https://colab.research.google.com/github/FTi130/pinns_shelve/blob/master/Navier_Stokes_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F

import scipy.io


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
class Net(nn.Module):
    def __init__(self, X, Y, T, u, v):
      super().__init__() # TODO Check if needed

      # self.fc1 = nn.Linear(3, 10)
      # self.fc2 = nn.Linear(20,20)
      # self.fc3 = nn.Linear(20,20)
      # self.fc4 = nn.Linear(20,20)
      # self.fc5 = nn.Linear(20,20)
      # self.fc6 = nn.Linear(20,20)
      # self.fc7 = nn.Linear(20,20)
      # self.fc8 = nn.Linear(20,20)

      # self.output_layer = nn.Linear(20,2)

      self.x = torch.tensor(X, dtype=torch.float32, requires_grad=True)
      self.y = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
      self.t = torch.tensor(T, dtype=torch.float32, requires_grad=True)

      self.u = torch.tensor(u, dtype=torch.float32)
      self.v = torch.tensor(v, dtype=torch.float32)

      #null vector to test against f and g:
      self.null = torch.zeros((self.x.shape[0], 1))

      # initialize network:
      self.network()

      self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr=1, max_iter=200000, max_eval=50000,
                                          history_size=50, tolerance_grad=1e-05, tolerance_change=0.5 * np.finfo(float).eps,
                                          line_search_fn="strong_wolfe")

      self.mse = nn.MSELoss()

      #loss
      self.ls = 0

      #iteration number
      self.iter = 0




    # def forward(self, x, y, t):

    #   inputs = torch.hstack((x,y,t))

    #   layer1_out = torch.sigmoid(self.fc1(inputs))
    #   layer2_out = torch.sigmoid(self.fc2(layer1_out))
    #   layer3_out = torch.sigmoid(self.fc3(layer2_out))
    #   layer4_out = torch.sigmoid(self.fc4(layer3_out))
    #   layer5_out = torch.sigmoid(self.fc5(layer4_out))
    #   layer6_out = torch.sigmoid(self.fc6(layer5_out))
    #   layer7_out = torch.sigmoid(self.fc7(layer6_out))
    #   layer8_out = torch.sigmoid(self.fc8(layer7_out))

    #   output_layer_out = torch.sigmoid(layer8_out)

    #   return output_layer_out


    def function(self, x, y, t):

      results = self.net(x ,y, t)

      psi, p = results[:, 0:1], results[:,1:2]

      nu = 0.01

      u = torch.autograd.grad(psi.sim(), y, create_graph=True)[0]
      v = -1.*torch.autograd.grad(psi.sim(), x, create_graph=True)[0]

      # U direction
      u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0] # Derivative over x
      u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]   # Second derivative over x
      u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0] # Derivative over y
      u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0] # Second derivative over y

      u_t = torch.autograd.grad(u.sum(), t, create_graph=True) # Time component


      # V direction
      v_x = torch.autograd.grad(v.sum(), x, create_graph=True)[0]
      v_xx = torch.autograd.grad(v_x.sum(), x, create_graph=True)[0]
      v_y = torch.autograd.grad(v.sum(), y, create_graph=True)[0]
      v_yy = torch.autograd.grad(v_y.sum(), y, create_graph=True)[0]

      v_t = torch.autograd.grad(v.sum(), t, create_graph=True) # Time component



      p_x = torch.autograd.grad(p.sum(), x, create_graph=True)[0]
      p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0]


      # Function itself

      f = u_t + u*u_x + p_x - nu*(u_xx + u_yy)
      g = v_t + v*v_y + p_y - nu*(v_xx + v_yy)

      return u, v, p, f, g

    def closure(self):
      # reset gradients to zero:
      self.optimizer.zero_grad()

      # u, v, p, g and f predictions:
      u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = self.function(self.x, self.y, self.t)

      # calculate losses
      u_loss = self.mse(u_prediction, self.u)
      v_loss = self.mse(v_prediction, self.v)
      f_loss = self.mse(f_prediction, self.null)
      g_loss = self.mse(g_prediction, self.null)
      self.ls = u_loss + v_loss + f_loss +g_loss

      # derivative with respect to net's weights:
      self.ls.backward()

      self.iter += 1
      if not self.iter % 1:
          print('Iteration: {:}, Loss: {:0.6f}'.format(self.iter, self.ls))

      return self.ls

    def train(self):

      # training loop
      self.net.train()
      self.optimizer.step(self.closure)

    def network(self):

        self.net = nn.Sequential(
            nn.Linear(3, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 2))





In [5]:
net = Net()

In [7]:
net.to(device)

Net(
  (fc1): Linear(in_features=3, out_features=10, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=20, bias=True)
  (fc6): Linear(in_features=20, out_features=20, bias=True)
  (fc7): Linear(in_features=20, out_features=20, bias=True)
  (fc8): Linear(in_features=20, out_features=20, bias=True)
  (output_layer): Linear(in_features=20, out_features=2, bias=True)
)

In [8]:
# mse_cost_function = nn.MSELoss()

In [9]:
# optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# from re import U


# def function(x, y, t, net):
#   results = net(x ,y, t)

#   psi, p = results[:, 0:1], results[:,1:2]

#   nu = 0.01

#   u = torch.autograd.grad(psi.sim(), y, create_graph=True)[0]
#   v = -1.*torch.autograd.grad(psi.sim(), x, create_graph=True)[0]

#   # U direction
#   u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0] # Derivative over x
#   u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]   # Second derivative over x
#   u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0] # Derivative over y
#   u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0] # Second derivative over y

#   u_t = torch.autograd.grad(u.sum(), t, create_graph=True) # Time component


#   # V direction
#   v_x = torch.autograd.grad(v.sum(), x, create_graph=True)[0]
#   v_xx = torch.autograd.grad(v_x.sum(), x, create_graph=True)[0]
#   v_y = torch.autograd.grad(v.sum(), y, create_graph=True)[0]
#   v_yy = torch.autograd.grad(v_y.sum(), y, create_graph=True)[0]

#   v_t = torch.autograd.grad(v.sum(), t, create_graph=True) # Time component



#   p_x = torch.autograd.grad(p.sum(), x, create_graph=True)[0]
#   p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0]


#   # Function itself

#   f = u_t + u*u_x + p_x - nu*(u_xx + u_yy)
#   g = v_t + v*v_y + p_y - nu*(v_xx + v_yy)

#   return u, v, p, f, g


In [ ]:
# From file



```
# From file
```



In [12]:
N_train = 5000

In [15]:
data = scipy.io.loadmat('cylinder_wake.mat')

In [16]:
U_star = data['U_star']  # N x 2 x T
P_star = data['p_star']  # N x T
t_star = data['t']  # T x 1
X_star = data['X_star']  # N x 2

In [17]:
N = X_star.shape[0]
T = t_star.shape[0]

In [18]:
x_test = X_star[:, 0:1]
y_test = X_star[:, 1:2]
p_test = P_star[:, 0:1]
u_test = U_star[:, 0:1, 0]
t_test = np.ones((x_test.shape[0], x_test.shape[1]))

In [19]:
# Rearrange Data
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

In [20]:
UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = P_star  # N x T

In [21]:
x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

In [22]:
u = UU.flatten()[:, None]  # NT x 1
v = VV.flatten()[:, None]  # NT x 1
p = PP.flatten()[:, None]  # NT x 1

In [23]:
# Training Data
idx = np.random.choice(N * T, N_train, replace=False)
x_train = x[idx, :]
y_train = y[idx, :]
t_train = t[idx, :]
u_train = u[idx, :]
v_train = v[idx, :]

In [36]:
pinn = Net(x_train, y_train, t_train, u_train, v_train)
pinn.net.load_state_dict(torch.load('model.pt'))
pinn.net.eval()

<ipython-input-36-0d0b4c270bc7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pinn.net.load_state_dict(torch.load('model.pt'))


EOFError: Ran out of input

In [32]:
x_test = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
y_test = torch.tensor(y_test, dtype=torch.float32, requires_grad=True)
t_test = torch.tensor(t_test, dtype=torch.float32, requires_grad=True)

In [37]:
u_out, v_out, p_out, f_out, g_out = pinn.function(x_test, y_test, t_test)

u_plot = p_out.data.cpu().numpy()
u_plot = np.reshape(u_plot, (50, 100))

fig, ax = plt.subplots()

plt.contourf(u_plot, levels=30, cmap='jet')
plt.colorbar()

TypeError: Sequential.forward() takes 2 positional arguments but 4 were given

In [30]:
def animate(i):
    ax.clear()
    u_out, v_out, p_out, f_out, g_out = pinn.function(x_test, y_test, i*t_test)
    u_plot = p_out.data.cpu().numpy()
    u_plot = np.reshape(u_plot, (50, 100))
    cax = ax.contourf(u_plot, levels=20, cmap='jet')
    plt.xlabel(r'$x$')
    plt.xlabel(r'$y$')
    plt.title(r'$p(x,\; y, \; t)$')

In [31]:
# Call animate method
ani = animation.FuncAnimation(fig, animate, 20, interval=1, blit=False)
#ani.save('p_field_lbfgs.gif')
#plt.close()
# Display the plot
plt.show()

NameError: name 'fig' is not defined